In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import matplotlib
import pandas
import numpy
print (matplotlib.__version__)
print (pandas.__version__)
print (numpy.__version__)

1.5.3
0.18.1
1.11.1


In [3]:
#Reading ratings file:
df = pd.read_csv('/home/serge/Desktop/Kanye/Hubert/Memoir/data/data/data.csv')

In [4]:
n_users = df.UserId.unique().shape[0]
n_jobs = df.JobId.unique().shape[0]
print (' Total Number of users = ' + str(n_users) + ' | Total Number of jobs = ' + str(n_jobs) )
n_users

 Total Number of users = 2500 | Total Number of jobs = 150


2500

In [5]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(df,test_size=0.25)

train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_jobs))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_jobs))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [6]:
# Memory-Based Collaborative Filtering

In [7]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [8]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [9]:
# To get the size of rating's data
#print (rating.shape)
# To get the firts five users!
#rating.head()
#len(rating.UserId)

In [10]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [11]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [12]:
#train_data_matrix,test_data_matrix,
train_data_matrix[:1,:2]

array([[ 0.,  4.]])

In [13]:
### Compute the cosine similarity
## A distance metric commonly used in Recommender system is 
## cosine simmilarity and uses rating as a n-dimensional matrix
## and the similarity is calculated based on the angle between these vectors.

In [14]:
print(item_similarity[:4, :4])
print(user_similarity[:4, :4])

[[  1.11022302e-16   7.07424310e-01   7.08169392e-01   6.94329010e-01]
 [  7.07424310e-01  -4.44089210e-16   6.84166107e-01   6.69188929e-01]
 [  7.08169392e-01   6.84166107e-01   0.00000000e+00   6.46455501e-01]
 [  6.94329010e-01   6.69188929e-01   6.46455501e-01   0.00000000e+00]]
[[  1.11022302e-16   7.33029899e-01   7.39628028e-01   7.49548955e-01]
 [  7.33029899e-01   0.00000000e+00   6.59296086e-01   7.05643969e-01]
 [  7.39628028e-01   6.59296086e-01   0.00000000e+00   6.85955522e-01]
 [  7.49548955e-01   7.05643969e-01   6.85955522e-01   0.00000000e+00]]


In [15]:
data_sims = pd.DataFrame(user_similarity)
data_sims.ix[:1, :1]

,0,1
0,1.110223e-16,0.73303
1,7.330299e-01,0.00000


In [16]:
#data_sims.ix[:,:1]
#data_sims.ix[1]
#data_sims[jobs_data]

In [17]:
print 'User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix))
print 'Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix))

User-based CF RMSE: 2.59388682566
Item-based CF RMSE: 2.59443528736


In [18]:
# def get_top_2(jobs_data):
#    p = data_sims[jobs_data].apply(lambda row: np.sum(row), axis=1)
#    p = p.order(ascending=False)
#    return p.index[p.index.isin(jobs_data)==False][:2] 
#print get_top_2()
 

In [19]:
# Let us check the computational time
%timeit pairwise_distances(train_data_matrix, metric='cosine')

1 loop, best of 3: 1.12 s per loop


In [20]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'job':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [21]:
# Model-based Collaborative Filtering

In [22]:
sparsity=round(1.0-len(df)/float(n_users*n_jobs),3)
print 'The sparsity level of MovieLens100K is ' +  str(sparsity*100) + '%'

The sparsity level of MovieLens100K is 46.7%


In [23]:
# with the calculated simalarity matrix now we can use it to predict tyhe non rated jobs offers!

In [24]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print 'User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix))

User-based CF MSE: 2.69743505618


In [25]:
item_prediction

array([[ 1.23351553,  1.20927818,  1.21596616, ...,  1.20041009,
         1.20360237,  1.23602052],
       [ 1.03478591,  1.02060941,  1.03324697, ...,  1.03565111,
         1.03061732,  1.03835643],
       [ 1.09251875,  1.09370567,  1.09169475, ...,  1.09088825,
         1.07405898,  1.09384994],
       ..., 
       [ 1.23137234,  1.26057708,  1.23144747, ...,  1.25878189,
         1.24396102,  1.26398779],
       [ 1.34598612,  1.31535353,  1.30876076, ...,  1.34429356,
         1.345894  ,  1.3121887 ],
       [ 0.85169667,  0.86153389,  0.86573124, ...,  0.8627491 ,
         0.84591458,  0.86543903]])

In [26]:
%timeit predict(train_data_matrix, item_similarity, type='job')
%timeit predict(train_data_matrix, user_similarity, type='user')

10 loops, best of 3: 106 ms per loop
1 loop, best of 3: 1.51 s per loop


In [27]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [28]:
print 'User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix))
print 'Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix))

User-based CF RMSE: 2.59388682566
Item-based CF RMSE: 2.59443528736


In [29]:
sparsity=round(1.0-len(df)/float(n_users*n_jobs),3)
print 'The sparsity level of WutikoData is ' +  str(sparsity*100) + '%'

The sparsity level of WutikoData is 46.7%


In [30]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k = 20)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)
print 'User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix))

User-based CF MSE: 2.69743505618


In [31]:
## Content-Boosted Collaborative Filtering:
# This approach uses a content-based predictor to enhance existing user data,
# and then provides personalized suggestions through collaborative filtering
# Content-Boosted Collaborative Filtering, performs better than a pure content-based predictor, pure
# collaborative filter, and a naive hybrid approach.

In [32]:
# Model Evaluation
# The metrics for evaluating the accuracy of a prediction algorithm can be divided into two main categories: 
# statistical accuracy metrics and decision-support metrics.
# 1. Statistical accuracy metrics evaluate the accu-
# racy of a predictor by comparing predicted values with user-
#provided values. To measure statistical accuracy we use the
# mean absolute error (MAE) metric — defined as the average
# absolute difference between predicted ratings and actual rat-
# ings. In our experiments we computed the MAE on the test
# set for each user, and then averaged over the set of test users

In [33]:

# 2.Decision-support accuracy measures how well predic-
# tions help users select high-quality items. We use Re-
# ceiver Operating Characteristic (ROC) sensitivity to mea-
# sure decision-support accuracy. A predictor can be treated as a filter, where predicting a high rating for an item is
# equivalent to accepting the item, and predicting a low rating
# is equivalent to rejecting the item. The ROC sensitivity is
# given by the area under the ROC curve — a curve that plots
# sensitivity versus 1-specificity for a predictor. Sensitivity is
# defined as the probability that a good item is accepted by the
# filter; and specificity is defined as the probability that a bad
# item is rejected by the filter. We consider an item good if the
# user gave it a rating of 4 or above, otherwise we consider the
# item bad.